# Advent of Code 2023

## Day 1

### Part 1

In [3]:
with open('inputs/day1.txt') as f:
    s = f.read()[:-1]

def parse(n):
    return int(n[0] + n[-1])

sum([parse([c for c in n if c.isnumeric()]) for n in s.split('\n')])

54634

### Part 2

In [4]:
numeral_map = {'one':'1',
               'two':'2',
               'three':'3',
               'four':'4',
               'five':'5',
               'six':'6',
               'seven':'7',
               'eight':'8',
               'nine':'9'}
numerals = list(numeral_map.keys()) + list(numeral_map.values())

def parse_numeral(n):
    return numeral_map[n] if n in numeral_map else n


def extract_number(s):
    indices_left = {n:s.find(n) for n in numerals if s.find(n) >= 0}
    indices_right = {n:s.rfind(n) for n in numerals if s.find(n) >= 0}
    leftmost = parse_numeral(min(indices_left,key=indices_left.get))
    rightmost = parse_numeral(max(indices_right,key=indices_right.get))
    return int(leftmost + rightmost)


sum([extract_number(n) for n in s.split('\n')])

53855

## Day 2

### Part 1

In [1]:
with open('inputs/day2.txt') as f:
    s = f.read()[:-1]

def parse_round(round):
    return {cubes.split(' ')[1]: int(cubes.split(' ')[0]) for cubes in round.split(', ')}


def parse_game(game):
    return [parse_round(round) for round in game.split(': ')[1].split('; ')]


def max_color_value(game, color):
    return (color, max([round.get(color,0) for round in game]))


def max_values(game):
    return dict(map(lambda c: max_color_value(game,c), ['red', 'green', 'blue']))


def is_valid(game,reds,greens,blues):
    maxes = max_values(game)
    return maxes.get('red',0) <= reds and maxes.get('green',0) <= greens and maxes.get('blue',0) <= blues


games = [parse_game(game) for game in s.split('\n')]
sum([i+1 for i,game in enumerate(games) if is_valid(game,12,13,14)])

2593

### Part 2

In [3]:
from functools import reduce

def game_power(game):
    return reduce(lambda a,b: a*b, max_values(game).values())


sum(map(game_power, games))

54699

## Day 3

### Part 1

In [114]:
from functools import reduce

with open('inputs/day3.txt') as f:
    s = f.read()[:-1].split('\n')

symbols = [(x,y) for y in range(len(s)) for x in range(len(s[0])) if not (s[y][x] == '.' or s[y][x].isnumeric())]
numbers = [(x,y) for y in range(len(s)) for x in range(len(s[0])) if s[y][x].isnumeric()]
neighbours = [(-1,-1), (-1,0), (-1,1), (0,-1), (0,1), (1,-1), (1,0), (1,1)]

def explore(reachable, current, target):
    (x,y) = current
    neighborhood = [(x+xd,y+yd) for (xd,yd) in neighbours]
    for coord in neighborhood:
        if coord in target and coord not in reachable:
            reachable.add(coord)
            explore(reachable, coord, target)
    return reachable


reachable = reduce(lambda acc,sym: explore(acc, sym, numbers), symbols, set())
number_starts = [(x,y) for (x,y) in reachable if (x-1,y) not in reachable]

def read_number(current, acc, reachable):
    (x,y) = current
    if current in reachable:
        return read_number((x+1,y), acc*10+int(s[y][x]), reachable)
    return acc


sum(map(lambda x: read_number(x,0,reachable), number_starts))

521515

### Part 2

In [116]:
symbols = [(x,y) for y in range(len(s)) for x in range(len(s[0])) if s[y][x] == '*']
reachable = {sym: explore(set(), sym, numbers) for sym in symbols}
numbers = list(reduce(lambda acc, v: acc.union(v), reachable.values(), set()))
number_starts = [(x,y) for (x,y) in numbers if (x-1,y) not in numbers]
reachable_number_starts = {sym: [n for n in reachable[sym] if n in number_starts] for sym in reachable}
gears = [reduce(lambda a,b: a*b, map(lambda x: read_number(x,0,numbers), num_starts), 1) 
         for num_starts in reachable_number_starts.values() if len(num_starts) == 2]

sum(gears)

69527306

## Day 4

### Part 1

In [39]:
with open('inputs/day4.txt') as f:
    s = f.read()[:-1]

input = [line.replace('  ', ' ').split(': ')[1].split(' | ') for line in s.split('\n')]
cards = [(list(map(int,card[0].split(' '))), list(map(int,card[1].split(' ')))) for card in input]
winning = [(len([x for x in card[1] if x in card[0]])) for card in cards]
sum([2 ** (card-1) for card in winning if card > 0])

21485

### Part 2

In [40]:
card_count = {i: 1 for i in range(len(cards))}
for i in range(len(cards)):
    for j in range(1, winning[i]+1):
        card_count[i+j] += card_count[i]

sum(card_count.values())

11024379

## Day 5

### Part 1

In [2]:
from functools import reduce

with open('inputs/day5.txt') as f:
    s = f.read()[:-1]

seeds = [int(seed) for seed in s.split('\n\n')[0].split(' ')[1:]]
maps = [[tuple(int(n) for n in m.split(' ')) for m in sec.split('\n')[1:]] for sec in s.split('\n\n')[1:]]

def attempt_map(n, map):
    destination_start, source_start, range_length = map
    if n >= source_start and n < source_start + range_length:
        return (destination_start + n - source_start, True)
    else:
        return (n, False)

def map_level(n,mapping):
    match = False
    i = -1
    while not match and i < len(mapping) - 1:
        i += 1
        result, match = attempt_map(n, mapping[i])
    return (result, i)


def map_full(n, levels):
    return reduce(lambda acc,level: map_level(acc[0],level), levels, (n,None))


min([map_full(seed, maps)[0] for seed in seeds])

484023871

### Part 2

In [3]:
from math import ceil

seed_ranges = [(seeds[i], seeds[i+1]) for i in range(0, len(seeds), 2)]
lowest_start = min([map_full(seed_range[0], maps) for seed_range in seed_ranges])

def find_buckets(seed_range, levels, buckets):
    (start, length) = seed_range
    if length <= 1:
        return buckets
    (result_1, last_map_1) = map_full(start, levels)
    (result_2, last_map_2) = map_full(start+length-1, levels)
    if last_map_1 not in buckets or buckets[last_map_1] > result_1:
        buckets[last_map_1] = result_1
    if last_map_2 not in buckets or buckets[last_map_2] > result_2:
        buckets[last_map_2] = result_2
    if last_map_1 != last_map_2:
        h1 = find_buckets((start, length//2), levels, buckets)
        h2 = find_buckets((start+length//2, ceil(length/2)), levels, buckets)
        for b in h1:
            if b not in buckets or buckets[b] > h1[b]:
                buckets[b] = h1[b]
        for b in h2:
            if b not in buckets or buckets[b] > h2[b]:
                buckets[b] = h2[b]
    return buckets


min(reduce(lambda acc,sr: find_buckets(sr,maps,acc), seed_ranges, {}).values())

46294175

## Day 6

### Part 1

In [34]:
from functools import reduce

with open('inputs/day6.txt') as f:
    s = f.read()[:-1]

[time, distance] = [[int(i) for i in ' '.join(line.split()).split(': ')[1].split(' ')] for line in s.split('\n')]

def distance_after(max_time, press_time):
    return (max_time-press_time)*press_time


reduce(lambda acc,i: acc*len([press for press in range(time[i]+1) if distance_after(time[i], press) > distance[i]]), range(len(time)), 1)

1624896

### Part 2

In [35]:
[time, distance] = [int(''.join(line.split(':')[1].split())) for line in s.split('\n')]
len([press for press in range(time+1) if distance_after(time, press) > distance])

32583852

## Day 7

### Part 1

In [89]:
with open('inputs/day7.txt') as f:
    s = f.read()[:-1].split('\n')


def count_chars(hand):
    chars = {}
    for c in hand:
        chars[c] = chars.get(c,0)+1
    return chars


def is_five_of_a_kind(hand):
    return 5 in count_chars(hand).values()


def is_four_of_a_kind(hand):
    return 4 in count_chars(hand).values()


def is_full_house(hand):
    values = count_chars(hand).values()
    return 3 in values and 2 in values


def is_three_of_a_kind(hand):
    values = count_chars(hand).values()
    return 3 in values and 2 not in values


def is_two_pairs(hand):
    values = count_chars(hand).values()
    return len([i for i in values if i == 2]) == 2


def is_pair(hand):
    values = count_chars(hand).values()
    return len([i for i in values if i == 2]) == 1 and 3 not in values


def is_high(hand):
    values = count_chars(hand).values()
    return len([i for i in values if i == 1]) == 5


types = [is_five_of_a_kind, is_four_of_a_kind, is_full_house, is_three_of_a_kind, is_two_pairs, is_pair, is_high]


def transform_hand(hand):
    return hand.replace('K', 'B').replace('Q','C').replace('J','D').replace('T','E') \
    .replace('9', 'F').replace('8','G').replace('7','H').replace('6','I').replace('5', 'J') \
    .replace('4','K').replace('3','L').replace('2','M').replace('1','N')


def normalize_hand(hand, bid):
    hand_t = transform_hand(hand)
    for i in range(len(types)):
        if types[i](hand_t):
            return ' '.join([str(i), hand_t, str(bid)])

hands = [normalize_hand(line.split(' ')[0], line.split(' ')[1]) for line in s]
hands.sort(reverse=True)

sum([(i+1)*int(hands[i].split(' ')[-1]) for i in range(len(hands))])

249483956

### Part 2

In [90]:
def transform_hand(hand):
    return hand.replace('K', 'B').replace('Q','C').replace('J','O').replace('T','E') \
    .replace('9', 'F').replace('8','G').replace('7','H').replace('6','I').replace('5', 'J') \
    .replace('4','K').replace('3','L').replace('2','M').replace('1','N')


def jokers(hand):
    return hand.count('O')


def count_chars(hand):
    chars = {}
    for c in hand:
        if c != 'O':
            chars[c] = chars.get(c,0)+1
    return chars


def is_five_of_a_kind(hand):
    joker_count = jokers(hand)
    return (5-jokers(hand) in count_chars(hand).values()) or joker_count == 5


def is_four_of_a_kind(hand):
    return 4-jokers(hand) in count_chars(hand).values()


def is_full_house(hand):
    values = count_chars(hand).values()
    joker_count = jokers(hand)
    return (joker_count == 0 and 3 in values and 2 in values) or \
    (joker_count == 1 and (3 in values or len([i for i in values if i == 2]) == 2))


def is_three_of_a_kind(hand):
    values = count_chars(hand).values()
    joker_count = jokers(hand)
    return (joker_count == 0 and 3 in values and 2 not in values) or \
    (joker_count == 1 and len([i for i in values if i == 2]) == 1) or \
    (joker_count == 2 and len([i for i in values if i == 1]) == 3)


def is_two_pairs(hand):
    values = count_chars(hand).values()
    return (len([i for i in values if i == 2]) == 2)


def is_pair(hand):
    values = count_chars(hand).values()
    joker_count = jokers(hand)
    return (joker_count == 0 and len([i for i in values if i == 2]) == 1 and 3 not in values) or \
    (joker_count == 1 and len([i for i in values if i == 1]) == 4)


def is_high(hand):
    values = count_chars(hand).values()
    return len([i for i in values if i == 1]) == 5


def normalize_hand(hand, bid):
    hand_t = transform_hand(hand)
    for i in range(len(types)):
        if types[i](hand_t):
            return ' '.join([str(i), hand_t, str(bid)])


types = [is_five_of_a_kind, is_four_of_a_kind, is_full_house, is_three_of_a_kind, is_two_pairs, is_pair, is_high]
hands = [normalize_hand(line.split(' ')[0], line.split(' ')[1]) for line in s]
hands.sort(reverse=True)

sum([(i+1)*int(hands[i].split(' ')[-1]) for i in range(len(hands))])

252137472

## Day 8

### Part 1

In [4]:
with open('inputs/day8.txt') as f:
    s = f.read()[:-1]

instructions = [0 if inst == 'L' else 1 for inst in s.split('\n')[0]]
nodes = {line.split(' = ')[0]: tuple(line.split(' = ')[-1][1:-1].split(', ')) for line in s.split('\n\n')[1].split('\n')}

current = 'AAA'
steps = 0
while current != 'ZZZ':
    current = nodes[current][instructions[steps % len(instructions)]]
    steps += 1

steps

12737

### Part 2

In [11]:
from functools import reduce
from math import lcm

def find_loop(start, instrtuctions, nodes):
    i = 0
    current = start
    visited = []
    ends = set()
    while (current, i % len(instructions)) not in visited:
        visited.append((current, i % len(instructions)))
        current = nodes[current][instructions[i % len(instructions)]]
        i += 1
        if current[-1] == 'Z':
            ends.add(i)
    loop_start = visited.index((current, i % len(instructions)))
    loop_length = i - loop_start
    return (loop_length, ends)


current_nodes = [node for node in nodes if node[-1] == 'A']
loops = [find_loop(start, instructions, nodes)[0] for start in current_nodes]
reduce(lambda acc,x: lcm(acc,x), loops, 1) # No further steps are necessary due to the input being a special case

9064949303801

## Day 9

### Part 1

In [48]:
with open('inputs/day9.txt') as f:
    s = f.read()[:-1].split('\n')

sequences = [[int(i) for i in line.split(' ')] for line in s]

def next_seq(seq):
    return [seq[i+1]-seq[i] for i in range(len(seq)-1)]


def is_last(seq):
    seq_set = set(seq)
    return (len(seq_set) == 1 and 0 in seq_set) or len(seq_set) == 0


def predict_next(seq):
    acc = 0
    while not is_last(seq):
        acc += seq[-1]
        seq = next_seq(seq)
    return acc


sum([predict_next(seq) for seq in sequences])

1882395907

### Part 2

In [62]:
def all_subsequences(seq):
    seqs = [seq]
    while not is_last(seqs[-1]):
        seqs.append(next_seq(seqs[-1]))
    return seqs


def predict_previous(seq):
    all = all_subsequences(seq)
    prev = 0
    for i in range(1,len(all)+1):
        current = all[-i][0] - prev
        prev = current
    return current


sum([predict_previous(seq) for seq in sequences])

1005

 ## Day 10

### Part 1

In [21]:
with open('inputs/day10.txt') as f:
    s = f.read()[:-1]

grid = s.split('\n')
neighbours = {'|': {(0,-1), (0,1)},
              '-': {(-1,0), (1,0)},
              'L': {(0,-1), (1,0)},
              'J': {(0,-1), (-1,0)},
              '7': {(0,1), (-1,0)},
              'F': {(0,1), (1,0)},
              'S': {(-1,0), (1,0), (0,-1), (0,1)},
              '.': {}}

for y,line in enumerate(grid):
    if 'S' in line:
        y0 = y
        x0 = line.index('S')

distances = {}
queue = {(x0,y0,0)}


def connections(x,y):
    return neighbours[grid[y][x]]


def connects_to(x0,y0,xd,yd):
    return (xd,yd) in connections(x0,y0) and (-xd,-yd) in connections(x0+xd, y0+yd)


def look_around(x,y):
    return [(x+xd,y+yd) for (xd,yd) in connections(x,y) if connects_to(x,y,xd,yd)]


def visit_neighbours(x,y,d):
    if (x,y) in distances and d >= distances[(x,y)]:
        return
    distances[(x,y)] = d
    connecting = look_around(x,y)
    for (x1,y1) in connecting:
        if (x1,y1) not in distances or distances[(x1,y1)] > d+1:
            queue.add((x1,y1,d+1))


while len(queue) > 0:
    x,y,d = queue.pop()
    visit_neighbours(x,y,d)


(xf,yf) = max(distances,key=distances.get)
distances[(xf,yf)]

6640

### Part 2

In [22]:
grid = [' '.join(list(y)) for y in s.split('\n')]
for i in range(len(grid)-1,-1,-1):
    grid.insert(i+1,' '*len(grid[0]))

neighbours = {'|': {(0,-2), (0,2)},
              '-': {(-2,0), (2,0)},
              'L': {(0,-2), (2,0)},
              'J': {(0,-2), (-2,0)},
              '7': {(0,2), (-2,0)},
              'F': {(0,2), (2,0)},
              'S': {(-2,0), (2,0), (0,-2), (0,2)},
              '.': {}}

for y,line in enumerate(grid):
    if 'S' in line:
        y0 = y
        x0 = line.index('S')


def connects_to(x0,y0,xd,yd):
    connects = (xd,yd) in connections(x0,y0) and (-xd,-yd) in connections(x0+xd, y0+yd)
    if connects:
        grid[y0+yd//2] = grid[y0+yd//2][:x0+xd//2] + 'x' + grid[y0+yd//2][x0+xd//2+1:]
    return connects


distances = {}
queue = {(x0,y0,0)}
while len(queue) > 0:
    x,y,d = queue.pop()
    visit_neighbours(x,y,d)

loop = [''.join(['.' if ((x,y) not in distances and grid[y][x] != 'x' and x % 2 == 0 and y % 2 == 0) else grid[y][x] \
                 for x in range(len(grid[0]))]) for y in range(len(grid))]

def expand(x0,y0):
    queue = {(x0,y0)}
    visited = set()
    while len(queue) > 0:
        (x,y) = queue.pop()
        if x >= 0 and x < len(grid[0]) and y >= 0 and y < len(grid) and (loop[y][x] == '.' or loop[y][x] == ' ') and (x,y) not in visited:
            visited.add((x,y))
            queue.add((x-1,y))
            queue.add((x+1,y))
            queue.add((x,y-1))
            queue.add((x,y+1))
    return visited

outside = expand(0,0)
inside = [''.join([' ' if (x,y) in outside else loop[y][x] for x in range(len(loop[0]))]) for y in range(len(loop))]

sum([line.count('.') for line in inside])

411

## Day 11

### Part 1

In [78]:
with open('inputs/day11.txt') as f:
    s = f.read()[:-1]


def transpose(matrix):
    return [''.join([matrix[y][x] for y in range(len(matrix))]) for x in range(len(matrix[0]))]


space = s.split('\n')
space_t = transpose(space)
empty_y = {y for y in range(len(space)) if '#' not in space[y]}
empty_x = {x for x in range(len(space_t)) if '#' not in space_t[x]}
galaxies = [(x,y) for x in range(len(space[0])) for y in range(len(space)) if space[y][x] == '#']


def distance(g1,g2,empty_value):
    (x1,y1) = g1
    (x2,y2) = g2
    acc = abs(x1-x2) + abs(y1-y2)
    for x in empty_x:
        if x in range(min(x1,x2), max(x1,x2)):
            acc += empty_value-1
    for y in empty_y:
        if y in range(min(y1,y2), max(y1,y2)):
            acc += empty_value-1
    return acc


acc = 0
for i in range(len(galaxies)):
    for j in range(i+1, len(galaxies)):
        acc += distance(galaxies[i], galaxies[j],2)

acc

9370588

### Part 2

In [79]:
acc = 0
for i in range(len(galaxies)):
    for j in range(i+1, len(galaxies)):
        acc += distance(galaxies[i], galaxies[j],1000000)

acc

746207878188

## Day 12

### Part 1

In [298]:
from functools import cache

with open('inputs/day12.txt') as f:
    s = f.read()[:-1]

@cache
def count_solutions(record, broken_blocks, solutions):
    if len(record) == 0 and len(broken_blocks) == 0:
        return 1
    if len(record) == 0 and len(broken_blocks) > 0:
        return 0
    if record[0] == '.':
        return count_solutions(record[1:], broken_blocks, solutions)
    if record[0] == '#':
        if len(broken_blocks) == 0 or len(record) < broken_blocks[0] or '.' in record[:broken_blocks[0]] or \
           (len(record) > broken_blocks[0] and record[broken_blocks[0]] == '#'):
            return 0
        else:
            return count_solutions('.' + record[broken_blocks[0]+1:], broken_blocks[1:], solutions)
    return count_solutions('#' + record[1:], broken_blocks, solutions+1) + count_solutions('.' + record[1:], broken_blocks, solutions+1)


springs = [(line.split(' ')[0], tuple([int(n) for n in line.split(' ')[1].split(',')])) for line in s.split('\n')]
sum([count_solutions(spring[0],spring[1],0) for spring in springs])

7204

### Part 2

In [299]:
def unfold(spring):
    (record, broken_blocks) = spring
    return ('?'.join([record]*5),broken_blocks*5)

unfolded_springs = [unfold(spring) for spring in springs]
sum([count_solutions(spring[0],spring[1],0) for spring in unfolded_springs])

1672318386674

## Day 13

### Part 1

In [89]:
with open('inputs/day13.txt') as f:
    s = f.read()[:-1]

def transpose(matrix):
    return [''.join([matrix[y][x] for y in range(len(matrix))]) for x in range(len(matrix[0]))]


def check_reflection(pattern,y):
    size = min(y, len(pattern)-y)
    return pattern[y-size:y] == pattern[y:y+size][::-1] 


def find_horizontal_reflection(pattern):
    for y in range(1,len(pattern)):
        if check_reflection(pattern,y):
            return y
    return 0


def find_vertical_reflection(pattern):
    pattern_t = transpose(pattern)
    for x in range(1,len(pattern_t)):
        if check_reflection(pattern_t,x):
            return x
    return 0
    

def find_reflection_index(pattern):
    return (find_vertical_reflection(pattern), find_horizontal_reflection(pattern))


def reflection_value(pattern):
    (ref_x, ref_y) = find_reflection_index(pattern)
    return 100*ref_y + ref_x


patterns = [pattern.split('\n') for pattern in s.split('\n\n')]
sum(map(lambda pattern: reflection_value(pattern), patterns))

26957

### Part 2

In [124]:
def flip(pattern,x,y):
    replacement = '.' if pattern[y][x] == '#' else '#'
    return [pattern[yy][:x] + replacement + pattern[yy][x+1:] if yy == y else pattern[yy] for yy in range(len(pattern))]

def find_horizontal_reflection_except(pattern,ex):
    for y in range(1,len(pattern)):
        if check_reflection(pattern,y) and y != ex:
            return y
    return 0


def find_vertical_reflection_except(pattern,ex):
    pattern_t = transpose(pattern)
    for x in range(1,len(pattern_t)):
        if check_reflection(pattern_t,x) and x != ex:
            return x
    return 0
    

def find_reflection_index_except(pattern,ex_x,ex_y):
    return (find_vertical_reflection_except(pattern,ex_x), find_horizontal_reflection_except(pattern,ex_y))


def reflection_value_smudged(pattern):
    (ref_x_0, ref_y_0) = find_reflection_index(pattern)
    for y in range(len(pattern)):
        for x in range(len(pattern[0])):
            cleaned = flip(pattern,x,y)
            (ref_x, ref_y) = find_reflection_index_except(cleaned, ref_x_0, ref_y_0)
            if ref_x == ref_x_0:
                ref_x = 0
            if ref_y == ref_y_0:
                ref_y = 0
            if ref_x + ref_y != 0:
                return 100*ref_y + ref_x


sum(map(lambda pattern: reflection_value_smudged(pattern), patterns))

42695

## Day 14

### Part 1

In [94]:
with open('inputs/day14.txt') as f:
    s = f.read()[:-1].split('\n')

min_x = 0
min_y = 0
max_x = len(s[0])-1
max_y = len(s)-1

static = {(x,y) for y in range(max_y+1) for x in range(max_x+1) if s[y][x] == '#'}
rolling = {(x,y) for y in range(max_y+1) for x in range(max_x+1) if s[y][x] == 'O'}


def discover_stuck(rolling, direction):
    (x_d,y_d) = direction
    stuck = {(x,y) for (x,y) in rolling if (x+x_d) < min_x or (y+y_d) < min_y or (x+x_d) > max_x or (y+y_d) > max_y or (x+x_d,y+y_d) in static}
    added = len(stuck)
    while added != 0:
        also_stuck = {(x,y) for (x,y) in rolling if (x+x_d,y+y_d) in stuck and (x,y) not in stuck}
        added = len(also_stuck)
        stuck.update(also_stuck)
    return stuck


def roll_once(rolling, direction):
    (x_d,y_d) = direction
    stuck = discover_stuck(rolling, direction)
    free = {(x+x_d,y+y_d) for (x,y) in rolling if (x,y) not in stuck}
    new_rolling = stuck.union(free)
    return (new_rolling, rolling != new_rolling)


def full_roll(rolling, direction):
    (rolling_new, changed) = roll(rolling, direction)
    while changed:
        (rolling_new, changed) = roll(rolling_new, direction)
    return rolling_new


def load(rolling):
    return sum([max_y+1-y for (_,y) in rolling])

rolling_new = full_roll(rolling, (0,-1))
load(rolling_new)

108144

### Part 2

In [95]:
cache = {}

def to_tuple(rolling):
    return tuple(sorted(list(rolling)))


def cycle(rolling,i):
    rolling_t = to_tuple(rolling)
    if rolling_t in cache:
        return cache[rolling_t]
    rolling = full_roll(rolling, (0,-1))
    rolling = full_roll(rolling, (-1,0))
    rolling = full_roll(rolling, (0,1))
    rolling = full_roll(rolling, (1,0))
    cache[rolling_t] = (rolling,i)
    return (rolling,None)

(rolling_new,cache_hit) = cycle(rolling, 0)
cycles = 1000000000
i = 1
while i < cycles:
    (rolling_new, first_seen) = cycle(rolling_new,i)
    if first_seen is not None:
        loop_length = i-first_seen
        i = first_seen + loop_length*((cycles-first_seen)//loop_length)
    i += 1

load(rolling_new)

108404